In [1]:
import torch
import torch.nn as nn

In [2]:
import torchvision

In [3]:
import random
import time
import os
import sys

In [4]:
network = 'alexnet'
batch_size = 1
iterations = 10

net = torchvision.models.alexnet()
is_gpu_available = False

In [5]:
if torch.cuda.is_available():
    print ("INFO: GPU is available, hence switching to gpu computation.")
    is_gpu_available = True
else:
    print ("INFO: GPU is not available, using CPU..")

INFO: GPU is available, hence switching to gpu computation.


In [6]:
inp = torch.randn(batch_size, 3, 224, 224)
if is_gpu_available:
    inp = inp.cuda()
    net = net.cuda()
    
target = torch.arange(batch_size)
if is_gpu_available:
    target = target.to('cuda:0')
    
param_copy = net.parameters()

In [8]:
print (net)

AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (classifier): Sequential(
    (0): Dropout(p=0.5)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
    (2): ReLU(inplace)
    (3): Dropout(p=0.5)
    (4): Linear(in_features=4096, out_feature

In [9]:
sub_modules = net.__dict__['_modules']
print (sub_modules)

OrderedDict([('features', Sequential(
  (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
  (1): ReLU(inplace)
  (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (4): ReLU(inplace)
  (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (7): ReLU(inplace)
  (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (9): ReLU(inplace)
  (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (11): ReLU(inplace)
  (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
)), ('classifier', Sequential(
  (0): Dropout(p=0.5)
  (1): Linear(in_features=9216, out_features=4096, bias=True)
  (2): ReLU(inplace)
  (3): Dropout(p=0.5)
  (4): Linear(in_features=4096, out_features=4096, bias=True)
  (5): ReLU(inp

In [36]:
def getLayers(module, layer_info):
    sub_modules = module.__dict__['_modules']
    count = 0
    for name, sub_module in sub_modules.items():
        if sub_module is None or isinstance(sub_module, nn.Module) is False:
            break
        if isinstance(sub_module, nn.Container) or isinstance(sub_module, nn.Sequential):
            getLayers(sub_module, layer_info)
        else:
            layer_info.append(sub_module)
            print (sub_module.__class__)

layer_info = []
getLayers(net, layer_info)
print (len(layer_info))

<class 'torch.nn.modules.conv.Conv2d'>
<class 'torch.nn.modules.activation.ReLU'>
<class 'torch.nn.modules.conv.Conv2d'>
<class 'torch.nn.modules.activation.ReLU'>
<class 'torch.nn.modules.pooling.MaxPool2d'>
<class 'torch.nn.modules.conv.Conv2d'>
<class 'torch.nn.modules.activation.ReLU'>
<class 'torch.nn.modules.conv.Conv2d'>
<class 'torch.nn.modules.activation.ReLU'>
<class 'torch.nn.modules.pooling.MaxPool2d'>
<class 'torch.nn.modules.conv.Conv2d'>
<class 'torch.nn.modules.activation.ReLU'>
<class 'torch.nn.modules.conv.Conv2d'>
<class 'torch.nn.modules.activation.ReLU'>
<class 'torch.nn.modules.conv.Conv2d'>
<class 'torch.nn.modules.activation.ReLU'>
<class 'torch.nn.modules.pooling.MaxPool2d'>
<class 'torch.nn.modules.conv.Conv2d'>
<class 'torch.nn.modules.activation.ReLU'>
<class 'torch.nn.modules.conv.Conv2d'>
<class 'torch.nn.modules.activation.ReLU'>
<class 'torch.nn.modules.conv.Conv2d'>
<class 'torch.nn.modules.activation.ReLU'>
<class 'torch.nn.modules.pooling.MaxPool2d'>


In [11]:
for i in range(1):
    layer = layer_info[0]
    print(layer.__class__)
    x = torch.randn(1, 3, 224, 224).to('cuda:0')
    print(layer)
    output = layer(x)
    output_size = output.size()
    grad_output = torch.randn(output_size[0], output_size[1], output_size[2], output_size[3]).cuda()
    output.backward(grad_output)

<class 'torch.nn.modules.conv.Conv2d'>
Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))


In [24]:
def generate_time(layer, x, iter=10):
    torch.cuda.synchronize()
    start_time = time.time()
    for j in range(1):
        if isinstance(layer, nn.Linear):
            print ("Linear layer detected... ")
        output = layer(x)
        output_size = output.size()
        #grad_output = torch.randn(output_size[0], output_size[1], output_size[2], output_size[3]).cuda()
        #output.backward(grad_output)
    torch.cuda.synchronize()
    elapsed_time = time.time() - start_time
    print ("Elapsed time is : {}".format(elapsed_time))
    return elapsed_time, output_size

In [29]:
x = torch.randn(1, 3, 224, 224).cuda()
output_sizes = {}
layer_data = {}
net_layer_data = {}
num_linear_layer = 0
for i in range(len(layer_info)):
    layer_data['layer_num'] = i
    layer = layer_info[i]
    if i != 0:
        prev_layer_info = net_layer_data[i - 1]
        prev_output_size = prev_layer_info['output_size']
        print(prev_output_size)
        if (len(prev_output_size) == 4):
            x = torch.randn(output_size[0], output_size[1], output_size[2], output_size[3]).cuda()
        elif (len(prev_output_size) == 2):
            x = torch.randn(output_size[0], output_size[1]).cuda()
        elif (len(prev_output_size) == 3):
            x = torch.randn(output_size[0], output_size[1], output_size[2]).cuda()
        if (isinstance(layer, nn.Linear) and  num_linear_layer == 0):
            x = x.view(-1, output_size[1] * output_size[2] * output_size[3]).cuda()
            num_linear_layer = num_linear_layer + 1
        print(x.size())
            
    print ("------------------------ Layer Num {} --------------------------------".format(i))
    print(layer)
    elapsed_time, output_size = generate_time(layer, x)
    layer_data['elapsed_time'] = elapsed_time
    layer_data['input_size'] = x.size()
    layer_data['output_size'] = output_size
    net_layer_data[i] = layer_data

------------------------ Layer Num 0 --------------------------------
Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
Elapsed time is : 0.0009970664978027344
torch.Size([1, 64, 55, 55])
torch.Size([1, 64, 55, 55])
------------------------ Layer Num 1 --------------------------------
ReLU(inplace)
Elapsed time is : 0.0009984970092773438
torch.Size([1, 64, 55, 55])
torch.Size([1, 64, 55, 55])
------------------------ Layer Num 2 --------------------------------
MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
Elapsed time is : 0.0
torch.Size([1, 64, 27, 27])
torch.Size([1, 64, 27, 27])
------------------------ Layer Num 3 --------------------------------
Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
Elapsed time is : 0.002028226852416992
torch.Size([1, 192, 27, 27])
torch.Size([1, 192, 27, 27])
------------------------ Layer Num 4 --------------------------------
ReLU(inplace)
Elapsed time is : 0.0
torch.Size([1, 192, 27, 

In [40]:
### Test with other network.
net = torchvision.models.vgg16()
print (net)
if is_gpu_available:
    net = net.cuda()
layer_info = []
getLayers(net, layer_info)
print (len(layer_info))
print (layer_info)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (17): Conv2d

In [41]:
x = torch.randn(1, 3, 224, 224).cuda()
output_sizes = {}
layer_data = {}
net_layer_data = {}
num_linear_layer = 0
for i in range(len(layer_info)):
    layer_data['layer_num'] = i
    layer = layer_info[i]
    if i != 0:
        prev_layer_info = net_layer_data[i - 1]
        prev_output_size = prev_layer_info['output_size']
        print(prev_output_size)
        if (len(prev_output_size) == 4):
            x = torch.randn(output_size[0], output_size[1], output_size[2], output_size[3]).cuda()
        elif (len(prev_output_size) == 2):
            x = torch.randn(output_size[0], output_size[1]).cuda()
        elif (len(prev_output_size) == 3):
            x = torch.randn(output_size[0], output_size[1], output_size[2]).cuda()
        if (isinstance(layer, nn.Linear) and  num_linear_layer == 0):
            x = x.view(-1, output_size[1] * output_size[2] * output_size[3]).cuda()
            num_linear_layer = num_linear_layer + 1
            
    print ("------------------------ Layer Num {} --------------------------------".format(i))
    print(x.size())
    print(layer)
    elapsed_time, output_size = generate_time(layer, x)
    layer_data['elapsed_time'] = elapsed_time
    layer_data['input_size'] = x.size()
    layer_data['output_size'] = output_size
    net_layer_data[i] = layer_data

------------------------ Layer Num 0 --------------------------------
torch.Size([1, 3, 224, 224])
Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
Elapsed time is : 0.005044698715209961
torch.Size([1, 64, 224, 224])
------------------------ Layer Num 1 --------------------------------
torch.Size([1, 64, 224, 224])
ReLU(inplace)
Elapsed time is : 0.002032756805419922
torch.Size([1, 64, 224, 224])
------------------------ Layer Num 2 --------------------------------
torch.Size([1, 64, 224, 224])
Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
Elapsed time is : 0.008017778396606445
torch.Size([1, 64, 224, 224])
------------------------ Layer Num 3 --------------------------------
torch.Size([1, 64, 224, 224])
ReLU(inplace)
Elapsed time is : 0.002034425735473633
torch.Size([1, 64, 224, 224])
------------------------ Layer Num 4 --------------------------------
torch.Size([1, 64, 224, 224])
MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mod